# Вебинар 2. Бейзлайны и детерминированные алгоритмы item-item

![recsys_types.png](attachment:recsys_types.png)

In [1]:
!pip install implicit==0.4.4

[Implicit](https://implicit.readthedocs.io/en/latest/quickstart.html) - очень быстрая и 
эффективная библиотека для рекоммендаций

Основные фичи:
    - Cython под капотом - высокая скорость
    - Множество приближенных алгоритмов - быстрее, чем оригинальные
    - Содежрит большинство популярных алгоритмов
    - Есть алгоритмы ранжирования
    - Поиск похожих товаров / юзеров
    - Есть возможность пересчета "холодного" юзера "на лету"
    - Возможность фильтровать товары при рекомендациях (Например, не рекомендовать женские товары мужчинам)
    - Есть метрики качества

# Подготовка данных

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [3]:
data = pd.read_csv('data/retail_train.csv')
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [4]:
data['week_no'].nunique()

95

In [5]:
users, items, interactions = data.user_id.nunique(), data.item_id.nunique(), data.shape[0]

print('# users: ', users)
print('# items: ', items)
print('# interactions: ', interactions)

# users:  2499
# items:  89051
# interactions:  2396804


In [6]:
popularity = data.groupby('item_id')['sales_value'].sum().reset_index()
popularity.describe()

,item_id,sales_value
count,8.905100e+04,89051.000000
mean,5.115772e+06,83.458481
std,5.178973e+06,1628.715079
min,2.567100e+04,0.000000
25%,9.665830e+05,3.500000
50%,1.448516e+06,10.780000
75%,9.553042e+06,46.105000
max,1.802456e+07,467993.620000


**Note:**  
Еще есть данные по характеристикам товаров и пользователей. Они нам пригодятся через несколько вебинаров

In [7]:
item_features = pd.read_csv('data/product.csv')
item_features.head()

,PRODUCT_ID,MANUFACTURER,DEPARTMENT,BRAND,COMMODITY_DESC,SUB_COMMODITY_DESC,CURR_SIZE_OF_PRODUCT
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ


In [8]:
user_features = pd.read_csv('data/hh_demographic.csv')
user_features.head()

,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE_DESC,KID_CATEGORY_DESC,household_key
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16


### Train-test split

В рекомендательных системах корректнее использовать train-test split по времени, а не случайно  
Я возьму последние 3 недели в качестве теста

In [9]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [10]:
data_train.shape[0], data_test.shape[0]

(2278490, 118314)

# 1. Бейзлайны

Создадим датафрейм с покупками юзеров на тестовом датасете (последние 3 недели)

In [11]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [12]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 0 новых юзеров


### 1.1 Random recommendation

In [13]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [14]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

result.head(2)

CPU times: total: 5.11 s
Wall time: 5.13 s


,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6045528, 12604278, 5565069, 1190222, 46533]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[38552, 10311387, 13006971, 1065380, 864214]"


### 1.2 Popularity-based recommendation

In [15]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [16]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

CPU times: total: 281 ms
Wall time: 220 ms


,user_id,actual,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6045528, 12604278, 5565069, 1190222, 46533]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[38552, 10311387, 13006971, 1065380, 864214]","[6534178, 6533889, 1029743, 6534166, 1082185]"


### 1.3 Weighted random recommender

- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

*Пример*  
item_1 - 5, item_2 - 7, item_3 - 4  # / sum  
item_1 - 5 / 16, item_2 - 7 / 16, item_3 - 4 / 16

In [17]:
#Подготовка датафрейма с весами для функции

popular = data.groupby('item_id')['sales_value'].sum() #Собираем суммы для весов
popular += 0.0001 #Добавляем искусственно коэффициент малой точности, чтобы уйти от нулей в признаке, иначе получим inf:Division by zero
items_weights = np.log10(popular).to_frame().reset_index()
items_weights.rename(columns={"sales_value": "weight", }, inplace=True)
items_weights['weight'].sum() #Проверяем что веса суммируются
#Собираю входной датафрейм с весами полученных через десятичный логарифм из sales_value
#Вес = log(sales_sum товара) - логарифм по какому основанию имеется ввиду? (небольшое замечание)

99470.5282675824

In [18]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    items_weights['weight_1'] = items_weights['weight']/items_weights['weight'].sum()
    results = items_weights.drop(columns=['weight'])
    results.sort_values('weight_1', ascending=False, inplace=True)
    recs = results.head(n).item_id
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code

    return recs.tolist()

In [19]:
weighted_random_recommendation(items_weights, n=5)

[6534178, 6533889, 1029743, 6534166, 1082185]

### Выводы по бейзлайнам
- Фиксируют базовое качество;
- Бейзлайны могут быть фильтрами;
- Иногда бейзлайны лучше ML-модели

# 2. Детерминированные алгоритмы

## 2.1 Item-Item Recommender / ItemKNN

![cf_memory.png](attachment:cf_memory.png)

![user_item_matrix.png](attachment:user_item_matrix.png)

То, что именно находится в матрице user-item нужно определять из бизнес-логики

Варианты для нашего датасета(не исчерпывающий список):
    - Факт покупки (0 / 1)
    - Кол-во покупок (count)
    - Сумма покупки, руб
    - ...
    
**Детерминированные алгоритмы**:
    - Предсказывают те числа, которые стоят в матрице

**ML-алгоритмы (большинство)**:
    - В качестве *таргетов* "под капотом" принимают 0 и 1 (в ячейке не 0 -> таргет 1)
    - А абсолютные значения воспринимают как *веса ошибок*
    
*P.S.* На самом деле есть много трюков, как можно заполнять матрицу user-item.

**Как работает Item-Item Recommender**

![item_item_recommender.png](attachment:item_item_recommender.png)

*Шаг 1:* Ищем K ближайших товаров к товару  
*Шаг 2*: predict "скора" товара = среднему "скору" этого товара у его соседей  
*Шаг 3*: Сортируем товары по убыванию predict-ов score и берем топ-k

----
**(!) Важно** 
- У item-item алгоритмов большая сложность ($O(I^2 log(I))$ или $O(I^3)$, в зависимости от реализации 
- Если в датасете много item_id, то item-item модели ОЧЕНЬ долго предсказывают. Со всеми товарами predict на тесте ~2 часа
- Давайте возьмем из ~90к товаров только 5k самых популярных 

*P.S.*  Брать топ-Х популярных и рекомендовать только из них - очень популярная стратегия.   
*P.P.S.*  В рекомендательных системах много таких трюков. Что-то подобное в курсе вы увидите еще не раз

In [20]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [21]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [22]:
data_train.head(100)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1060,26985040735,1,9553288,1,8.49,315,0.00,1251,1,0.0,0.0
96,1351,26985052379,1,903230,1,0.99,447,-0.30,1955,1,0.0,0.0
97,744,26985165432,1,5978648,0,0.00,31582,0.00,1119,1,0.0,0.0
98,212,26985205886,1,822346,1,1.25,288,-0.34,1341,1,0.0,0.0


In [23]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 6666
data_train.head(100)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1060,26985040735,1,9553288,1,8.49,315,0.00,1251,1,0.0,0.0
96,1351,26985052379,1,903230,1,0.99,447,-0.30,1955,1,0.0,0.0
97,744,26985165432,1,6666,0,0.00,31582,0.00,1119,1,0.0,0.0
98,212,26985205886,1,822346,1,1.25,288,-0.34,1341,1,0.0,0.0


In [24]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix)


## Разряженность матрицы

In [25]:
user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100

5.33770796861036

In [26]:
# создаем словари мапинга между id бизнеса к строчному id матрицы

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

# Fit

In [27]:
%%time

model = ItemItemRecommender(K=13, num_threads=4) # K - кол-во билжайших соседей


model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: total: 3.28 s
Wall time: 1.62 s


In [28]:
recs

[(0, 78679.0),
 (3408, 72173.0),
 (2149, 60612.0),
 (301, 49790.0),
 (2308, 49269.0)]

In [29]:
[id_to_itemid[rec[0]] for rec in recs]

[6666, 1082185, 981760, 840361, 995242]

In [30]:
%%time

result['itemitem'] = result['user_id'].apply(lambda user_id: [id_to_itemid[rec[0]]  
                                                              for rec in model.recommend(userid=userid_to_id[user_id], 
                                                                user_items=sparse_user_item,   # на вход user-item matrix
                                                                N=5, 
                                                                filter_already_liked_items=False, 
                                                                filter_items=None, 
                                                                recalculate_user=True)
                                                                             ])

CPU times: total: 141 ms
Wall time: 121 ms


In [31]:
result.head(10)

,user_id,actual,random_recommendation,popular_recommendation,itemitem
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6045528, 12604278, 5565069, 1190222, 46533]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 995242, 840361]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[38552, 10311387, 13006971, 1065380, 864214]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 826249]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[959328, 965009, 994266, 955337, 966430]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 840361, 995242]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[17171513, 906483, 1117669, 8062667, 8014960]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 995242, 840361]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[2094016, 5740106, 8066552, 12810447, 1526340]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 995242, 840361]"
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8...","[880797, 949132, 1107075, 876102, 15973036]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 995242]"
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884...","[8357443, 6602312, 963566, 9676699, 858420]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 840361, 1098066]"
7,14,"[840601, 867293, 933067, 951590, 952408, 96569...","[982908, 1025057, 12647938, 13115505, 5995228]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 995242]"
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13...","[788330, 921451, 1010578, 15973009, 9526996]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 995242, 840361]"
9,16,"[1062973, 1082185, 13007710]","[1849083, 870877, 1044125, 13878848, 10118684]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 840361, 995242]"


### 4.2 Косинусное сходство и CosineRecommender

![cosine_similarity.png](attachment:cosine_similarity.png)

In [32]:
%%time

model = CosineRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: total: 3.67 s
Wall time: 1.68 s


In [33]:
[id_to_itemid[rec[0]] for rec in recs]

[1082185, 1029743, 1081177, 904360, 6034857]

In [34]:
%%time

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: total: 93.8 ms
Wall time: 94.7 ms


In [35]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6045528, 12604278, 5565069, 1190222, 46533]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 995242, 840361]","[1082185, 1029743, 1081177, 904360, 6034857]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[38552, 10311387, 13006971, 1065380, 864214]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 826249]","[1082185, 951590, 1044078, 9526410, 904360]"


### 4.3 TF-IDF взвешивание и TFIDFRecommender

![tf_idf.png](attachment:tf_idf.png)

Если 2 юзера оба купили очень популярный товар, то это еще не значит,что они похожи   
Если 2 юзера оба купили редкий товар, то они похожи

Занижаем вес популярных товаров при расчете расстояний между пользователями

In [36]:
%%time

model = TFIDFRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: total: 3.31 s
Wall time: 1.33 s


In [37]:
[id_to_itemid[rec[0]] for rec in recs]

[1082185, 981760, 1127831, 6666, 1098066]

In [38]:
%%time

result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=False, 
                                    recalculate_user=False)])

CPU times: total: 93.8 ms
Wall time: 89.8 ms


In [39]:
result.head(5)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6045528, 12604278, 5565069, 1190222, 46533]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 995242, 840361]","[1082185, 1029743, 1081177, 904360, 6034857]","[1082185, 981760, 1127831, 6666, 1098066]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[38552, 10311387, 13006971, 1065380, 864214]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 826249]","[1082185, 951590, 1044078, 9526410, 904360]","[1082185, 981760, 1098066, 826249, 6666]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[959328, 965009, 994266, 955337, 966430]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 840361, 995242]","[1082185, 951590, 994928, 1029743, 1044078]","[1082185, 981760, 1127831, 6666, 878996]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[17171513, 906483, 1117669, 8062667, 8014960]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 995242, 840361]","[1082185, 994928, 1044078, 1029743, 6034857]","[1082185, 981760, 1127831, 6666, 961554]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[2094016, 5740106, 8066552, 12810447, 1526340]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 995242, 840361]","[1082185, 854852, 951590, 1029743, 1044078]","[1082185, 981760, 6666, 1098066, 826249]"


### 4.4 Трюк

In [40]:
%%time

model = ItemItemRecommender(K=13, num_threads=4) # K - кол-во билжайших соседей


model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: total: 3.09 s
Wall time: 1.12 s


In [41]:
%%time

result['own_purchases'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[6666]], 
                                    recalculate_user=True)])

CPU times: total: 266 ms
Wall time: 140 ms


### 4.5 Измерим качество по precision@5

In [42]:
result.head(5)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6045528, 12604278, 5565069, 1190222, 46533]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 995242, 840361]","[1082185, 1029743, 1081177, 904360, 6034857]","[1082185, 981760, 1127831, 6666, 1098066]","[1082185, 981760, 995242, 840361, 1127831]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[38552, 10311387, 13006971, 1065380, 864214]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 826249]","[1082185, 951590, 1044078, 9526410, 904360]","[1082185, 981760, 1098066, 826249, 6666]","[1082185, 981760, 1098066, 826249, 995242]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[959328, 965009, 994266, 955337, 966430]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 840361, 995242]","[1082185, 951590, 994928, 1029743, 1044078]","[1082185, 981760, 1127831, 6666, 878996]","[1082185, 981760, 840361, 995242, 1127831]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[17171513, 906483, 1117669, 8062667, 8014960]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 995242, 840361]","[1082185, 994928, 1044078, 1029743, 6034857]","[1082185, 981760, 1127831, 6666, 961554]","[1082185, 981760, 995242, 840361, 1098066]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[2094016, 5740106, 8066552, 12810447, 1526340]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 995242, 840361]","[1082185, 854852, 951590, 1029743, 1044078]","[1082185, 981760, 6666, 1098066, 826249]","[1082185, 981760, 995242, 840361, 1098066]"


In [43]:
!python metrics.py

In [44]:
# Функции из 1-ого вебинара
import os, sys
    
from metrics import precision_at_k, recall_at_k

# Metrics

In [45]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0009:random_recommendation
0.1552:popular_recommendation
0.1532:itemitem
0.1728:cosine
0.139:tfidf
0.1663:own_purchases


# Articles

https://www.cs.umd.edu/~samir/498/Amazon-Recommendations.pdf

https://en.wikipedia.org/wiki/Collaborative_filtering

https://core.ac.uk/download/pdf/83041762.pdf



User-based and item-based CF explanation

https://medium.com/@cfpinela/recommender-systems-user-based-and-item-based-collaborative-filtering-5d5f375a127f